# Replication Notebook for "Fiscal and Monetary Policy with Heterogeneous Agents" by Auclert et al. (2024) 

#### Course: HANK in Macroeconomics | Supervisor: Selma Malmberg
#### Authors: Maximilian Stein & Gabriele Piantoni | Submitted on December 9th, 2025

This notebook will combine the calibrated household block from `Household_Solving.ipynb` and its parametric outcomes from `hh_parameter.json`, which then replicates Figure 2(a) and (b) from Auclert et al. (2024). Hence, it combines the three distinct household blocks for the HA, TA and RA models.